# Missing data imputation with Fedbiomed using MIWAE

In this notebook we show how to impute missing not at random (MAR) data in a federated setting using MIWAE (https://arxiv.org/abs/2006.12871). We will compare results of federated training using FedAvg, FedProx and FedCos with local results.

In [1]:
%load_ext autoreload
%autoreload 2

## Prepare the data

For this experiment we will use the breast cancer data from sklearn.

In [2]:
import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [3]:
from sklearn.model_selection import train_test_split

data_train, data_test, labels_train, labels_test = train_test_split(data, target, test_size=0.20, random_state=42)
df_data_train = pd.DataFrame(data_train)
N_train = len(df_data_train)
client_1, client_2, client_3 = np.split(df_data_train.sample(frac=1,random_state=42), \
                                        [int(.33*N_train), int(.66*len(df_data_train))])

Clients_data=[client_1, client_2, client_3]

# from each dataset we will remove randomly 50% of data
np.random.seed(1234)

# 50% of missing data for client 1, 30% for client 2, 60% for client 3
perc_miss_list = [0.5,0.3,0.6] 

Clients_missing = []
for perc,c in enumerate(Clients_data):
    perc_miss=perc_miss_list[perc]
    n = c.shape[0] # number of observations
    p = c.shape[1] # number of features
    xmiss = np.copy(c)
    xmiss = (xmiss - np.mean(xmiss,0))/np.std(xmiss,0)
    xmiss_flat = xmiss.flatten()
    miss_pattern = np.random.choice(n*p, np.floor(n*p*perc_miss).astype(np.int_),\
                                    replace=False)
    xmiss_flat[miss_pattern] = np.nan 
    xmiss = xmiss_flat.reshape([n,p]) # in xmiss, the missing values are represented by nans
    mask = np.isfinite(xmiss) # binary mask that indicates which values are missing
    Clients_missing.append(xmiss)

import os 
os.makedirs('clients_data', exist_ok=True) 
for i in range(len(Clients_missing)):
    pd.DataFrame(Clients_missing[i]).to_csv('clients_data/client_'+str(i+1)+'.csv',index=False)

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the nodes up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 1 (csv) to add client_1 dataset to the first node
  * Provide the correct tag by entering:  breast_cancer
  * Pick the folder where client_1 dataset has been saved
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node start`. Wait until you get `Starting task manager`. it means you are online.
4. Following the same procedure, you can create additional nodes for clients 2 and 3.

Check available clients:

In [4]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)
xx = req.list()
dataset_size = [xx[i][0]['shape'][1] for i in xx]
assert min(dataset_size)==max(dataset_size)
data_size = dataset_size[0]

2022-05-16 16:43:13,801 fedbiomed INFO - Component environment:
2022-05-16 16:43:13,802 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-05-16 16:43:13,840 fedbiomed INFO - Messaging researcher_cb04eb91-c6c6-4b99-aafb-70ef2f3fb7f0 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x129b45580>
2022-05-16 16:43:13,926 fedbiomed INFO - Listing available datasets in all nodes... 
2022-05-16 16:43:23,948 fedbiomed INFO - 
 Node: node_2ad05852-b939-4f31-8c03-b6905bf7497e | Number of Datasets: 1 
+---------------+-------------+-------------------+---------------+-----------+
| name          | data_type   | tags              | description   | shape     |
+===============+=============+===================+===============+===========+
| breast_cancer | csv         | ['breast_cancer'] | breast_cancer | [139, 13] |
+---------------+-------------+-------------------+---------------+-----------+

2022-05-16 16:43:23,950 fedbiomed INFO - 
 Nod

## Define an experiment model and parameters

Declare a torch.nn MIWAETrainingPlan class to send for training on the node

Note : write **only** the code to export in the following cell

In [5]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import numpy as np
import torch.distributions as td
import pandas as pd

from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from fedbiomed.common.constants import ProcessTypes

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MIWAETrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MIWAETrainingPlan, self).__init__(model_args)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        deps = ["from torchvision import datasets, transforms",
               "import torch.distributions as td",
               "import pandas as pd",
               "import numpy as np"]
        
        self.n_features=model_args['n_features']
        self.n_latent=model_args['n_latent']
        self.n_hidden=model_args['n_hidden']
        self.n_samples=model_args['n_samples']
        
        self.add_dependency(deps)
        
        # the encoder will output both the mean and the diagonal covariance
        self.encoder=nn.Sequential(
                        torch.nn.Linear(self.n_features, self.n_hidden),
                        torch.nn.ReLU(),
                        torch.nn.Linear(self.n_hidden, self.n_hidden),
                        torch.nn.ReLU(),
                        torch.nn.Linear(self.n_hidden, 2*self.n_latent),  
                        )
        # the decoder will output both the mean, the scale, 
        # and the number of degrees of freedoms (hence the 3*p)
        self.decoder = nn.Sequential(
                        torch.nn.Linear(self.n_latent, self.n_hidden),
                        torch.nn.ReLU(),
                        torch.nn.Linear(self.n_hidden, self.n_hidden),
                        torch.nn.ReLU(),
                        torch.nn.Linear(self.n_hidden, 3*self.n_features),  
                        )
        
        self.optimizer = torch.optim.Adam(list(self.encoder.parameters()) \
                                    + list(self.decoder.parameters()),lr=1e-3)
              
        self.encoder.apply(self.weights_init)
        self.decoder.apply(self.weights_init)
    
    def weights_init(self,layer):
        if type(layer) == nn.Linear: torch.nn.init.orthogonal_(layer.weight)
    
    def miwae_loss(self,iota_x,mask):
        batch_size = iota_x.shape[0]
        out_encoder = self.encoder(iota_x)
        # prior
        p_z = td.Independent(td.Normal(loc=torch.zeros(self.n_latent).to(self.device)\
                                       ,scale=torch.ones(self.n_latent).to(self.device)),1)
        
        q_zgivenxobs = td.Independent(td.Normal(loc=out_encoder[..., :self.n_latent],\
                                                scale=torch.nn.Softplus()\
                                                (out_encoder[..., self.n_latent:\
                                                             (2*self.n_latent)])),1)

        zgivenx = q_zgivenxobs.rsample([self.n_samples])
        zgivenx_flat = zgivenx.reshape([self.n_samples*batch_size,self.n_latent])

        out_decoder = self.decoder(zgivenx_flat)
        all_means_obs_model = out_decoder[..., :self.n_features]
        all_scales_obs_model = torch.nn.Softplus()(out_decoder[..., self.n_features:\
                                                               (2*self.n_features)]) + 0.001
        all_degfreedom_obs_model = torch.nn.Softplus()\
        (out_decoder[..., (2*self.n_features):(3*self.n_features)]) + 3

        data_flat = torch.Tensor.repeat(iota_x,[self.n_samples,1]).reshape([-1,1])
        tiledmask = torch.Tensor.repeat(mask,[self.n_samples,1])

        all_log_pxgivenz_flat = torch.distributions.StudentT\
        (loc=all_means_obs_model.reshape([-1,1]),\
         scale=all_scales_obs_model.reshape([-1,1]),\
         df=all_degfreedom_obs_model.reshape([-1,1])).log_prob(data_flat)
        all_log_pxgivenz = all_log_pxgivenz_flat.reshape([self.n_samples*batch_size,self.n_features])

        logpxobsgivenz = torch.sum(all_log_pxgivenz*tiledmask,1).reshape([self.n_samples,batch_size])
        logpz = p_z.log_prob(zgivenx)
        logq = q_zgivenxobs.log_prob(zgivenx)

        neg_bound = -torch.mean(torch.logsumexp(logpxobsgivenz + logpz - logq,0))

        return neg_bound

    def training_data(self,  batch_size = 48):
        
        df = pd.read_csv(self.dataset_path, sep=',', index_col=False)
        x_train = df.values
        x_mask = np.isfinite(x_train)
        # xhat_0: missing values are replaced by zeros. 
        #This x_hat0 is what will be fed to our encoder.
        xhat_0 = np.copy(x_train)
        xhat_0[np.isnan(x_train)] = 0
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        
        data_manager = DataManager(dataset=xhat_0 , target=x_mask , **train_kwargs)
        
        return data_manager
    
    def training_step(self, data, mask):
        self.encoder.zero_grad()
        self.decoder.zero_grad()
        loss = self.miwae_loss(iota_x = data,mask = mask)
        return loss

This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side. 
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.
* data `tags` to search nodes for training.
* total number of `rounds`.
If FedProx optimisation is requested, `fedprox_mu` parameter must be defined here. It also must be a float between XX and YY.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [6]:
h = 128 # number of hidden units in (same for all MLPs)
d = 10 # dimension of the latent space, we choose d=1 for visualisation purposes
K = 20 # number of IS during training

n_epochs=5

model_args = {'n_features':data_size, 'n_latent':d,'n_hidden':h,'n_samples':K}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'log_interval' : 1,
    'epochs': n_epochs, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

tags =  ['breast_cancer']
rounds = 30

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [7]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MIWAETrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-05-16 16:43:36,677 fedbiomed INFO - Searching dataset with data tags: ['breast_cancer'] for all nodes
2022-05-16 16:43:46,699 fedbiomed INFO - Node selected for training -> node_2ad05852-b939-4f31-8c03-b6905bf7497e
2022-05-16 16:43:46,700 fedbiomed INFO - Node selected for training -> node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc
2022-05-16 16:43:46,701 fedbiomed INFO - Node selected for training -> node_16062a6e-3512-4146-a89c-c379e90c9300
2022-05-16 16:43:46,707 fedbiomed INFO - Checking data quality of federated datasets...
2022-05-16 16:43:46,779 fedbiomed DEBUG - Model file has been saved: /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0000/my_model_66a4f368-7819-4844-a110-6a38c819655a.py
2022-05-16 16:43:47,068 fedbiomed DEBUG - upload (HTTP POST request) of file /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0000/my_model_66a4f368-7819-4844-a110-6a38c819655a.py successful, with status code 201
2022-05-16 

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [8]:
exp.run()

2022-05-16 16:43:47,848 fedbiomed INFO - Sampled nodes in round 0 ['node_2ad05852-b939-4f31-8c03-b6905bf7497e', 'node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc', 'node_16062a6e-3512-4146-a89c-c379e90c9300']
2022-05-16 16:43:47,858 fedbiomed INFO - Sending request 
					 To: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_cb04eb91-c6c6-4b99-aafb-70ef2f3fb7f0', 'job_id': '6f4f6206-9ca3-4f8d-ac6b-6005d7c9abec', 'training_args': {'test_ratio': 0.0, 'test_on_local_updates': False, 'test_on_global_updates': False, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'log_interval': 1, 'epochs': 5, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {'n_features': 13, 'n_latent': 10, 'n_hidden': 128, 'n_samples': 20}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/05/16/my_model_66a4f368-7819-4844-a110-6a38c819655a.py', 'params_url': 'http://localhos

2022-05-16 16:43:49,751 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 1 | Completed: 48/138 (33%) 
 					 Loss: 4.596571 
					 ---------
2022-05-16 16:43:49,763 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 4 | Completed: 48/133 (33%) 
 					 Loss: 8.940277 
					 ---------
2022-05-16 16:43:49,785 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 1 | Completed: 96/138 (67%) 
 					 Loss: 5.428924 
					 ---------
2022-05-16 16:43:49,789 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 4 | Completed: 96/133 (67%) 
 					 Loss: 10.730469 
					 ---------
2022-05-16 16:43:49,810 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 1 | Completed: 48/133 (33%) 
 					 Loss: 6.594474 
					 ---------
2022-05-16 16:43:49,840 fedbiomed INFO - TRAINING

2022-05-16 16:43:57,977 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_38500529-9796-433c-85c6-89a4e456aff3.pt successful, with status code 200
2022-05-16 16:43:57,993 fedbiomed INFO - Downloading model params after training on node_16062a6e-3512-4146-a89c-c379e90c9300 - from http://localhost:8844/media/uploads/2022/05/16/node_params_5575e69f-a8de-49fe-a83c-8a144cdc36b6.pt
2022-05-16 16:43:58,031 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_b25b9572-279b-414d-9770-557a5d3b5c42.pt successful, with status code 200
2022-05-16 16:43:58,035 fedbiomed INFO - Downloading model params after training on node_2ad05852-b939-4f31-8c03-b6905bf7497e - from http://localhost:8844/media/uploads/2022/05/16/node_params_79651145-307b-4d86-aaf5-2fc4fb937390.pt
2022-05-16 16:43:58,078 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_b0060c02-cbc5-4ad2-ba4d-a448d264f05d.pt successful, with status code 200
2022-05-16 16:43:58,092 fedbiomed INFO - Nodes that s

2022-05-16 16:43:58,912 fedbiomed INFO - INFO
					 NODE node_16062a6e-3512-4146-a89c-c379e90c9300
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x12f3d71f0>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'batch_size': 48, 'lr': 0.001, 'log_interval': 1, 'epochs': 5, 'dry_run': False, 'batch_maxnum': 100}
-----------------------------------------------------------------
2022-05-16 16:43:58,914 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 1 | Completed: 48/133 (33%) 
 					 Loss: 5.448351 
					 ---------
2022-05-16 16:43:58,917 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 1 | Completed: 96/133 (67%) 
 					 Loss: 5.273358 
					 ---------
2022-05-16 16:43:58,919 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 2 | Completed: 48/133 (33%) 
 					 

2022-05-16 16:44:00,374 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 5 | Completed: 96/133 (67%) 
 					 Loss: 4.848558 
					 ---------
2022-05-16 16:44:00,734 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 5 | Completed: 126/138 (100%) 
 					 Loss: 3.938922 
					 ---------
2022-05-16 16:44:00,736 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 5 | Completed: 111/133 (100%) 
 					 Loss: 4.925948 
					 ---------
2022-05-16 16:44:01,521 fedbiomed INFO - INFO
					 NODE node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2022-05-16 16:44:01,713 fedbiomed INFO - INFO
					 NODE node_16062a6e-3512-4146-a89c-c379e90c9300
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2022

...

2022-05-16 16:48:37,915 fedbiomed INFO - INFO
					 NODE node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x135461f70>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'batch_size': 48, 'lr': 0.001, 'log_interval': 1, 'epochs': 5, 'dry_run': False, 'batch_maxnum': 100}
-----------------------------------------------------------------
2022-05-16 16:48:37,932 fedbiomed INFO - WARNING
					 NODE node_2ad05852-b939-4f31-8c03-b6905bf7497e
					 MESSAGE: There is no test activated for the round. Please set flag for `test_on_global_updates`, `test_on_local_updates`, or both. Splitting dataset for testing will be ignored
-----------------------------------------------------------------
2022-05-16 16:48:37,959 fedbiomed INFO - INFO
					 NODE node_2ad05852-b939-4f31-8c03-b6905bf7497e
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed.node.history_monit

2022-05-16 16:48:38,629 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 4 | Completed: 96/133 (67%) 
 					 Loss: 1.428372 
					 ---------
2022-05-16 16:48:38,661 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 4 | Completed: 48/138 (33%) 
 					 Loss: 0.444056 
					 ---------
2022-05-16 16:48:38,686 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 4 | Completed: 111/133 (100%) 
 					 Loss: 2.006565 
					 ---------
2022-05-16 16:48:38,699 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 4 | Completed: 96/138 (67%) 
 					 Loss: -0.807398 
					 ---------
2022-05-16 16:48:38,711 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 4 | Completed: 96/133 (67%) 
 					 Loss: -1.048583 
					 ---------
2022-05-16 16:48:38,729 fedbiomed INFO - TRAIN

2022-05-16 16:48:48,018 fedbiomed DEBUG - researcher_cb04eb91-c6c6-4b99-aafb-70ef2f3fb7f0
2022-05-16 16:48:48,020 fedbiomed INFO - Sending request 
					 To: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_cb04eb91-c6c6-4b99-aafb-70ef2f3fb7f0', 'job_id': '6f4f6206-9ca3-4f8d-ac6b-6005d7c9abec', 'training_args': {'test_ratio': 0.0, 'test_on_local_updates': False, 'test_on_global_updates': False, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'log_interval': 1, 'epochs': 5, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {'n_features': 13, 'n_latent': 10, 'n_hidden': 128, 'n_samples': 20}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/05/16/my_model_66a4f368-7819-4844-a110-6a38c819655a.py', 'params_url': 'http://localhost:8844/media/uploads/2022/05/16/aggregated_params_fa49d85f-ce9f-4de9-8c53-81d3acf018ef.pt', 'model_class': 'MIWA

2022-05-16 16:48:48,622 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 3 | Completed: 126/138 (100%) 
 					 Loss: -0.493383 
					 ---------
2022-05-16 16:48:48,648 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 3 | Completed: 48/133 (33%) 
 					 Loss: 2.888206 
					 ---------
2022-05-16 16:48:48,699 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 3 | Completed: 96/133 (67%) 
 					 Loss: 1.594733 
					 ---------
2022-05-16 16:48:48,713 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 3 | Completed: 96/133 (67%) 
 					 Loss: -0.851085 
					 ---------
2022-05-16 16:48:48,732 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 3 | Completed: 111/133 (100%) 
 					 Loss: 2.127757 
					 ---------
2022-05-16 16:48:48,749 fedbiomed INFO - TRA

30

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [9]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()


List the training rounds :  dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

List the nodes for the last training round and their timings : 
	- node_2ad05852-b939-4f31-8c03-b6905bf7497e :    
		rtime_training=0.82 seconds    
		ptime_training=0.34 seconds    
		rtime_total=10.03 seconds
	- node_16062a6e-3512-4146-a89c-c379e90c9300 :    
		rtime_training=0.90 seconds    
		ptime_training=0.36 seconds    
		rtime_total=10.05 seconds
	- node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc :    
		rtime_training=0.98 seconds    
		ptime_training=0.37 seconds    
		rtime_total=10.10 seconds




,success,msg,dataset_id,node_id,params_path,params,timing
0,True,,dataset_682df8d3-2bfc-4473-b824-a0749a2158bf,node_2ad05852-b939-4f31-8c03-b6905bf7497e,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'encoder.0.weight': [[tensor(0.0575), tensor(...","{'rtime_training': 0.820694498000023, 'ptime_t..."
1,True,,dataset_038bafb0-c635-4c99-b222-dccee4419466,node_16062a6e-3512-4146-a89c-c379e90c9300,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'encoder.0.weight': [[tensor(0.0611), tensor(...","{'rtime_training': 0.8998911279999788, 'ptime_..."
2,True,,dataset_d5188f72-0351-45f6-ae53-42ef85fc6708,node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'encoder.0.weight': [[tensor(0.0635), tensor(...","{'rtime_training': 0.9762395679999827, 'ptime_..."


Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [10]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())


List the training rounds :  dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

Access the federated params for the last training round :
	- params_path:  /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0000/aggregated_params_9c00fd74-3d80-406c-8559-3f80575b78c4.pt
	- parameter data:  odict_keys(['encoder.0.weight', 'encoder.0.bias', 'encoder.2.weight', 'encoder.2.bias', 'encoder.4.weight', 'encoder.4.bias', 'decoder.0.weight', 'decoder.0.bias', 'decoder.2.weight', 'decoder.2.bias', 'decoder.4.weight', 'decoder.4.bias'])


## Run the experiment with FedProx

We repeat the federated training but using FedProx as aggregation scheme (starting from the second iteration).

In [11]:
# First round: simply FedAvg
training_args.update(fedprox_mu = 0.)

exp_fedprox = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MIWAETrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-05-16 16:49:30,214 fedbiomed INFO - Searching dataset with data tags: ['breast_cancer'] for all nodes
2022-05-16 16:49:40,230 fedbiomed INFO - Node selected for training -> node_16062a6e-3512-4146-a89c-c379e90c9300
2022-05-16 16:49:40,231 fedbiomed INFO - Node selected for training -> node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc
2022-05-16 16:49:40,232 fedbiomed INFO - Node selected for training -> node_2ad05852-b939-4f31-8c03-b6905bf7497e
2022-05-16 16:49:40,235 fedbiomed INFO - Checking data quality of federated datasets...
2022-05-16 16:49:40,253 fedbiomed DEBUG - Model file has been saved: /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0001/my_model_0ed2286b-24b5-460d-9e33-80b5462f55dd.py
2022-05-16 16:49:40,344 fedbiomed DEBUG - upload (HTTP POST request) of file /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0001/my_model_0ed2286b-24b5-460d-9e33-80b5462f55dd.py successful, with status code 201
2022-05-16 

In [12]:
exp_fedprox.run_once()

2022-05-16 16:49:40,567 fedbiomed INFO - Sampled nodes in round 0 ['node_16062a6e-3512-4146-a89c-c379e90c9300', 'node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc', 'node_2ad05852-b939-4f31-8c03-b6905bf7497e']
2022-05-16 16:49:40,567 fedbiomed INFO - Sending request 
					 To: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_cb04eb91-c6c6-4b99-aafb-70ef2f3fb7f0', 'job_id': '799e110d-39f0-49b8-b268-fca63480bad5', 'training_args': {'test_ratio': 0.0, 'test_on_local_updates': False, 'test_on_global_updates': False, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'log_interval': 1, 'epochs': 5, 'dry_run': False, 'batch_maxnum': 100, 'fedprox_mu': 0.0}, 'training': True, 'model_args': {'n_features': 13, 'n_latent': 10, 'n_hidden': 128, 'n_samples': 20}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/05/16/my_model_0ed2286b-24b5-460d-9e33-80b5462f55dd.py', 'params_url

2022-05-16 16:49:41,097 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 1 | Completed: 96/133 (67%) 
 					 Loss: 10.169772 
					 ---------
2022-05-16 16:49:41,102 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 2 | Completed: 96/133 (67%) 
 					 Loss: 6.693090 
					 ---------
2022-05-16 16:49:41,117 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 2 | Completed: 48/138 (33%) 
 					 Loss: 4.045870 
					 ---------
2022-05-16 16:49:41,128 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 1 | Completed: 111/133 (100%) 
 					 Loss: 9.665789 
					 ---------
2022-05-16 16:49:41,148 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 2 | Completed: 111/133 (100%) 
 					 Loss: 5.943799 
					 ---------
2022-05-16 16:49:41,152 fedbiomed INFO - TRAI

2022-05-16 16:49:50,609 fedbiomed INFO - Downloading model params after training on node_16062a6e-3512-4146-a89c-c379e90c9300 - from http://localhost:8844/media/uploads/2022/05/16/node_params_a062a67f-0558-40d8-b03c-3a43a17721d2.pt
2022-05-16 16:49:50,750 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_d73c8e11-d70b-45d5-bfc5-e9655d1c6ad7.pt successful, with status code 200
2022-05-16 16:49:50,771 fedbiomed INFO - Downloading model params after training on node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc - from http://localhost:8844/media/uploads/2022/05/16/node_params_855a7f7e-d76b-4882-b660-1935a4638c68.pt
2022-05-16 16:49:50,815 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_7c405db0-33ea-4934-a34f-dcac44545555.pt successful, with status code 200
2022-05-16 16:49:50,819 fedbiomed INFO - Downloading model params after training on node_2ad05852-b939-4f31-8c03-b6905bf7497e - from http://localhost:8844/media/uploads/2022/05/16/node_params_86cdaa46-5719-4e99-9f6

1

In [13]:
# Starting from the second round, FedProx is used with mu=0.1
training_args.update(fedprox_mu = 0.1)
exp_fedprox.run()

2022-05-16 16:49:51,140 fedbiomed INFO - Sampled nodes in round 1 ['node_16062a6e-3512-4146-a89c-c379e90c9300', 'node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc', 'node_2ad05852-b939-4f31-8c03-b6905bf7497e']
2022-05-16 16:49:51,141 fedbiomed INFO - Sending request 
					 To: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_cb04eb91-c6c6-4b99-aafb-70ef2f3fb7f0', 'job_id': '799e110d-39f0-49b8-b268-fca63480bad5', 'training_args': {'test_ratio': 0.0, 'test_on_local_updates': False, 'test_on_global_updates': False, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'log_interval': 1, 'epochs': 5, 'dry_run': False, 'batch_maxnum': 100, 'fedprox_mu': 0.0}, 'training': True, 'model_args': {'n_features': 13, 'n_latent': 10, 'n_hidden': 128, 'n_samples': 20}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/05/16/my_model_0ed2286b-24b5-460d-9e33-80b5462f55dd.py', 'params_url

2022-05-16 16:49:51,571 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 2 | Completed: 96/133 (67%) 
 					 Loss: 4.945250 
					 ---------
2022-05-16 16:49:51,611 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 2 | Completed: 111/133 (100%) 
 					 Loss: 5.401281 
					 ---------
2022-05-16 16:49:51,623 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 1 | Completed: 48/138 (33%) 
 					 Loss: 3.636859 
					 ---------
2022-05-16 16:49:51,634 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 1 | Completed: 96/138 (67%) 
 					 Loss: 3.979947 
					 ---------
2022-05-16 16:49:51,640 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 2 | Completed: 96/133 (67%) 
 					 Loss: 8.351855 
					 ---------
2022-05-16 16:49:51,653 fedbiomed INFO - TRAININ

2022-05-16 16:50:01,183 fedbiomed INFO - Downloading model params after training on node_16062a6e-3512-4146-a89c-c379e90c9300 - from http://localhost:8844/media/uploads/2022/05/16/node_params_28ba03c8-4b0c-48f6-8ab3-f233d28113e1.pt
2022-05-16 16:50:01,243 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_6002a732-ec21-444f-9988-34e1b146a7b8.pt successful, with status code 200
2022-05-16 16:50:01,257 fedbiomed INFO - Downloading model params after training on node_2ad05852-b939-4f31-8c03-b6905bf7497e - from http://localhost:8844/media/uploads/2022/05/16/node_params_0a0aa14e-1b5c-4f99-b0a5-40d110f4d778.pt
2022-05-16 16:50:01,291 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_abbd9ef1-bdb0-44f5-8ec0-8c4388e63d00.pt successful, with status code 200
2022-05-16 16:50:01,298 fedbiomed INFO - Downloading model params after training on node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc - from http://localhost:8844/media/uploads/2022/05/16/node_params_6abcbd36-cf84-4aee-92a

2022-05-16 16:50:01,742 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 1 | Completed: 48/138 (33%) 
 					 Loss: 3.308342 
					 ---------
2022-05-16 16:50:01,780 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 1 | Completed: 96/138 (67%) 
 					 Loss: 2.602001 
					 ---------
2022-05-16 16:50:01,814 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 1 | Completed: 48/133 (33%) 
 					 Loss: 4.169768 
					 ---------
2022-05-16 16:50:01,821 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 1 | Completed: 48/133 (33%) 
 					 Loss: 8.336164 
					 ---------
2022-05-16 16:50:01,830 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 1 | Completed: 126/138 (100%) 
 					 Loss: 3.884526 
					 ---------
2022-05-16 16:50:01,846 fedbiomed INFO - TRAININ

2022-05-16 16:50:02,443 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 5 | Completed: 111/133 (100%) 
 					 Loss: 5.463189 
					 ---------
2022-05-16 16:50:02,455 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 5 | Completed: 48/138 (33%) 
 					 Loss: 2.622719 
					 ---------
2022-05-16 16:50:02,477 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 5 | Completed: 111/133 (100%) 
 					 Loss: 3.302288 
					 ---------
2022-05-16 16:50:02,484 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 5 | Completed: 96/138 (67%) 
 					 Loss: 2.960745 
					 ---------
2022-05-16 16:50:02,631 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 5 | Completed: 126/138 (100%) 
 					 Loss: 1.834092 
					 ---------
2022-05-16 16:50:02,829 fedbiomed INFO - INF

...

2022-05-16 16:54:38,460 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 1 | Completed: 96/138 (67%) 
 					 Loss: 0.411776 
					 ---------
2022-05-16 16:54:38,486 fedbiomed INFO - WARNING
					 NODE node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc
					 MESSAGE: There is no test activated for the round. Please set flag for `test_on_global_updates`, `test_on_local_updates`, or both. Splitting dataset for testing will be ignored
-----------------------------------------------------------------
2022-05-16 16:54:38,493 fedbiomed INFO - INFO
					 NODE node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x136694040>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'batch_size': 48, 'lr': 0.001, 'log_interval': 1, 'epochs': 5, 'dry_run': False, 'batch_maxnum': 100, 'fedprox_mu': 0.0}
--------------------------------------------

2022-05-16 16:54:39,006 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 5 | Completed: 96/138 (67%) 
 					 Loss: 0.186945 
					 ---------
2022-05-16 16:54:39,016 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 4 | Completed: 96/133 (67%) 
 					 Loss: 1.157107 
					 ---------
2022-05-16 16:54:39,037 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 5 | Completed: 126/138 (100%) 
 					 Loss: -0.185877 
					 ---------
2022-05-16 16:54:39,050 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 5 | Completed: 111/133 (100%) 
 					 Loss: -0.299560 
					 ---------
2022-05-16 16:54:39,053 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 4 | Completed: 111/133 (100%) 
 					 Loss: 2.763859 
					 ---------
2022-05-16 16:54:39,113 fedbiomed INFO - T

2022-05-16 16:54:48,711 fedbiomed INFO - WARNING
					 NODE node_16062a6e-3512-4146-a89c-c379e90c9300
					 MESSAGE: There is no test activated for the round. Please set flag for `test_on_global_updates`, `test_on_local_updates`, or both. Splitting dataset for testing will be ignored
-----------------------------------------------------------------
2022-05-16 16:54:48,719 fedbiomed INFO - INFO
					 NODE node_2ad05852-b939-4f31-8c03-b6905bf7497e
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x13989c3d0>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'batch_size': 48, 'lr': 0.001, 'log_interval': 1, 'epochs': 5, 'dry_run': False, 'batch_maxnum': 100, 'fedprox_mu': 0.0}
-----------------------------------------------------------------
2022-05-16 16:54:48,726 fedbiomed INFO - INFO
					 NODE node_16062a6e-3512-4146-a89c-c379e90c9300
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed

2022-05-16 16:54:49,177 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 4 | Completed: 96/133 (67%) 
 					 Loss: -0.865423 
					 ---------
2022-05-16 16:54:49,198 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 4 | Completed: 48/133 (33%) 
 					 Loss: 2.013665 
					 ---------
2022-05-16 16:54:49,223 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 4 | Completed: 96/138 (67%) 
 					 Loss: 0.373784 
					 ---------
2022-05-16 16:54:49,250 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 4 | Completed: 96/133 (67%) 
 					 Loss: 2.783447 
					 ---------
2022-05-16 16:54:49,300 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 4 | Completed: 111/133 (100%) 
 					 Loss: -0.425241 
					 ---------
2022-05-16 16:54:49,302 fedbiomed INFO - TRAIN

29

## Run the experiment with FedCos

And finally we propose to use FedCos as well, which introduce an alternative penalization term with cosine similarity:

In [14]:
from fedbiomed.researcher.aggregators.fedcos import FedCos

del training_args['fedprox_mu'] # We can not perform simultaneously FedProx and FedCos

training_args.update(fedcos_mu = 0.01)

exp_fedcos = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MIWAETrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedCos(),
                 node_selection_strategy=None)

2022-05-16 16:54:59,038 fedbiomed INFO - Searching dataset with data tags: ['breast_cancer'] for all nodes
2022-05-16 16:55:09,057 fedbiomed INFO - Node selected for training -> node_16062a6e-3512-4146-a89c-c379e90c9300
2022-05-16 16:55:09,058 fedbiomed INFO - Node selected for training -> node_2ad05852-b939-4f31-8c03-b6905bf7497e
2022-05-16 16:55:09,059 fedbiomed INFO - Node selected for training -> node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc
2022-05-16 16:55:09,061 fedbiomed INFO - Checking data quality of federated datasets...
2022-05-16 16:55:09,080 fedbiomed DEBUG - Model file has been saved: /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0002/my_model_35f3aec9-e256-47b0-b271-7fc4cc83ad40.py
2022-05-16 16:55:09,165 fedbiomed DEBUG - upload (HTTP POST request) of file /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0002/my_model_35f3aec9-e256-47b0-b271-7fc4cc83ad40.py successful, with status code 201
2022-05-16 

In [15]:
exp_fedcos.run()

2022-05-16 16:55:09,354 fedbiomed INFO - Sampled nodes in round 0 ['node_16062a6e-3512-4146-a89c-c379e90c9300', 'node_2ad05852-b939-4f31-8c03-b6905bf7497e', 'node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc']
2022-05-16 16:55:09,355 fedbiomed INFO - Sending request 
					 To: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_cb04eb91-c6c6-4b99-aafb-70ef2f3fb7f0', 'job_id': 'eb394fbe-c782-480e-8df9-b9e876539015', 'training_args': {'test_ratio': 0.0, 'test_on_local_updates': False, 'test_on_global_updates': False, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'log_interval': 1, 'epochs': 5, 'dry_run': False, 'batch_maxnum': 100, 'fedcos_mu': 0.01}, 'training': True, 'model_args': {'n_features': 13, 'n_latent': 10, 'n_hidden': 128, 'n_samples': 20}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/05/16/my_model_35f3aec9-e256-47b0-b271-7fc4cc83ad40.py', 'params_url

2022-05-16 16:55:09,678 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 1 | Completed: 111/133 (100%) 
 					 Loss: 10.978781 
					 ---------
2022-05-16 16:55:09,709 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 2 | Completed: 48/133 (33%) 
 					 Loss: 6.204214 
					 ---------
2022-05-16 16:55:09,712 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 2 | Completed: 48/133 (33%) 
 					 Loss: 10.445449 
					 ---------
2022-05-16 16:55:09,716 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 2 | Completed: 48/138 (33%) 
 					 Loss: 4.677885 
					 ---------
2022-05-16 16:55:09,735 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 2 | Completed: 96/133 (67%) 
 					 Loss: 9.075247 
					 ---------
2022-05-16 16:55:09,747 fedbiomed INFO - TRAIN

2022-05-16 16:55:19,392 fedbiomed INFO - Downloading model params after training on node_16062a6e-3512-4146-a89c-c379e90c9300 - from http://localhost:8844/media/uploads/2022/05/16/node_params_86b0856e-3677-420d-8d06-47273f08a2da.pt
2022-05-16 16:55:19,433 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_cd1ee99b-e661-434d-813e-ab6b9d33504f.pt successful, with status code 200
2022-05-16 16:55:19,442 fedbiomed INFO - Downloading model params after training on node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc - from http://localhost:8844/media/uploads/2022/05/16/node_params_657f1a7c-1aa9-4452-996e-882df43d6de0.pt
2022-05-16 16:55:19,481 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_f23f3856-8111-43ad-86da-9eadf7f7a045.pt successful, with status code 200
2022-05-16 16:55:19,487 fedbiomed INFO - Downloading model params after training on node_2ad05852-b939-4f31-8c03-b6905bf7497e - from http://localhost:8844/media/uploads/2022/05/16/node_params_dc899b01-34f2-43e0-987

2022-05-16 16:55:20,590 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 1 | Completed: 48/133 (33%) 
 					 Loss: 5.494037 
					 ---------
2022-05-16 16:55:20,592 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 1 | Completed: 48/138 (33%) 
 					 Loss: 3.637454 
					 ---------
2022-05-16 16:55:20,605 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 1 | Completed: 48/133 (33%) 
 					 Loss: 9.033013 
					 ---------
2022-05-16 16:55:20,626 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 1 | Completed: 96/133 (67%) 
 					 Loss: 5.440269 
					 ---------
2022-05-16 16:55:20,648 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 1 | Completed: 96/133 (67%) 
 					 Loss: 8.425566 
					 ---------
2022-05-16 16:55:20,656 fedbiomed INFO - TRAINING 

2022-05-16 16:55:21,124 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 5 | Completed: 111/133 (100%) 
 					 Loss: 3.949698 
					 ---------
2022-05-16 16:55:21,130 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 5 | Completed: 96/133 (67%) 
 					 Loss: 7.900723 
					 ---------
2022-05-16 16:55:21,166 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 5 | Completed: 96/138 (67%) 
 					 Loss: 3.347586 
					 ---------
2022-05-16 16:55:21,174 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 5 | Completed: 111/133 (100%) 
 					 Loss: 5.614923 
					 ---------
2022-05-16 16:55:21,208 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 5 | Completed: 126/138 (100%) 
 					 Loss: 2.631977 
					 ---------
2022-05-16 16:55:21,447 fedbiomed INFO - INF

...

2022-05-16 17:00:00,173 fedbiomed INFO - INFO
					 NODE node_16062a6e-3512-4146-a89c-c379e90c9300
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x1305fd280>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'batch_size': 48, 'lr': 0.001, 'log_interval': 1, 'epochs': 5, 'dry_run': False, 'batch_maxnum': 100, 'fedcos_mu': 0.01}
-----------------------------------------------------------------
2022-05-16 17:00:00,190 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 1 | Completed: 48/138 (33%) 
 					 Loss: 0.576041 
					 ---------
2022-05-16 17:00:00,248 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 1 | Completed: 48/133 (33%) 
 					 Loss: 3.157123 
					 ---------
2022-05-16 17:00:00,250 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 1 | Completed: 48

2022-05-16 17:00:00,983 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 5 | Completed: 48/133 (33%) 
 					 Loss: 2.135071 
					 ---------
2022-05-16 17:00:00,998 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 5 | Completed: 96/133 (67%) 
 					 Loss: -0.538160 
					 ---------
2022-05-16 17:00:01,009 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 5 | Completed: 96/138 (67%) 
 					 Loss: 0.426078 
					 ---------
2022-05-16 17:00:01,027 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 5 | Completed: 111/133 (100%) 
 					 Loss: -0.437257 
					 ---------
2022-05-16 17:00:01,033 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 5 | Completed: 96/133 (67%) 
 					 Loss: 0.767645 
					 ---------
2022-05-16 17:00:01,064 fedbiomed INFO - TRAIN

2022-05-16 17:00:10,551 fedbiomed INFO - INFO
					 NODE node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x1366940d0>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'batch_size': 48, 'lr': 0.001, 'log_interval': 1, 'epochs': 5, 'dry_run': False, 'batch_maxnum': 100, 'fedcos_mu': 0.01}
-----------------------------------------------------------------
2022-05-16 17:00:10,567 fedbiomed INFO - WARNING
					 NODE node_16062a6e-3512-4146-a89c-c379e90c9300
					 MESSAGE: There is no test activated for the round. Please set flag for `test_on_global_updates`, `test_on_local_updates`, or both. Splitting dataset for testing will be ignored
-----------------------------------------------------------------
2022-05-16 17:00:10,583 fedbiomed INFO - INFO
					 NODE node_16062a6e-3512-4146-a89c-c379e90c9300
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed

2022-05-16 17:00:11,206 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 4 | Completed: 48/133 (33%) 
 					 Loss: 0.334744 
					 ---------
2022-05-16 17:00:11,209 fedbiomed INFO - TRAINING 
					 NODE_ID: node_2ad05852-b939-4f31-8c03-b6905bf7497e 
					 Epoch: 4 | Completed: 96/138 (67%) 
 					 Loss: 0.033629 
					 ---------
2022-05-16 17:00:11,246 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 4 | Completed: 96/133 (67%) 
 					 Loss: 0.722916 
					 ---------
2022-05-16 17:00:11,263 fedbiomed INFO - TRAINING 
					 NODE_ID: node_16062a6e-3512-4146-a89c-c379e90c9300 
					 Epoch: 4 | Completed: 96/133 (67%) 
 					 Loss: -0.391464 
					 ---------
2022-05-16 17:00:11,280 fedbiomed INFO - TRAINING 
					 NODE_ID: node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc 
					 Epoch: 4 | Completed: 111/133 (100%) 
 					 Loss: 2.629674 
					 ---------
2022-05-16 17:00:11,309 fedbiomed INFO - TRAINI

30

# Test and comparison to local training

## 1. Testing on an external dataset

First of all we are going to test the performance of the final federated model to impute missing data on a test dataset. To this extent we are going to remove randomly 50% of samples from the test dataset, `data_test`, defined at the beginning of this notebook.

In [16]:
# from the test dataset, we will remove randomly 50% of data
np.random.seed(1234)

perc_miss = 0.5 # 50% of missing data

n = data_test.shape[0] # number of observations
p = data_test.shape[1] # number of features
xfull = np.copy(data_test)
xfull = (xfull - np.mean(xfull,0))/np.std(xfull,0)
xmiss = np.copy(xfull)
xmiss_flat = xmiss.flatten()
miss_pattern = np.random.choice(n*p, np.floor(n*p*perc_miss).astype(np.int_),\
                                replace=False)
xmiss_flat[miss_pattern] = np.nan 
xmiss = xmiss_flat.reshape([n,p]) # in xmiss, the missing values are represented by nans
mask = np.isfinite(xmiss) # binary mask that indicates which values are missing
xhat_0 = np.copy(xmiss)
xhat_0[np.isnan(xmiss)] = 0
xhat = np.copy(xhat_0) # This will be out imputed data matrix

We define the MIWAE imputation routine:

In [17]:
def miwae_impute(encoder,decoder,iota_x,mask,d,L):
    
    p_z = td.Independent(td.Normal(loc=torch.zeros(d),scale=torch.ones(d)),1)
    
    batch_size = iota_x.shape[0]
    out_encoder = encoder(iota_x)
    q_zgivenxobs = td.Independent(td.Normal(loc=out_encoder[..., :d],scale=torch.nn.Softplus()(out_encoder[..., d:(2*d)])),1)

    zgivenx = q_zgivenxobs.rsample([L])
    zgivenx_flat = zgivenx.reshape([L*batch_size,d])

    out_decoder = decoder(zgivenx_flat)
    all_means_obs_model = out_decoder[..., :p]
    all_scales_obs_model = torch.nn.Softplus()(out_decoder[..., p:(2*p)]) + 0.001
    all_degfreedom_obs_model = torch.nn.Softplus()(out_decoder[..., (2*p):(3*p)]) + 3

    data_flat = torch.Tensor.repeat(iota_x,[L,1]).reshape([-1,1])
    tiledmask = torch.Tensor.repeat(mask,[L,1])

    all_log_pxgivenz_flat = torch.distributions.StudentT(loc=all_means_obs_model.reshape([-1,1]),scale=all_scales_obs_model.reshape([-1,1]),df=all_degfreedom_obs_model.reshape([-1,1])).log_prob(data_flat)
    all_log_pxgivenz = all_log_pxgivenz_flat.reshape([L*batch_size,p])

    logpxobsgivenz = torch.sum(all_log_pxgivenz*tiledmask,1).reshape([L,batch_size])
    logpz = p_z.log_prob(zgivenx)
    logq = q_zgivenxobs.log_prob(zgivenx)

    xgivenz = td.Independent(td.StudentT(loc=all_means_obs_model, scale=all_scales_obs_model, df=all_degfreedom_obs_model),1)

    imp_weights = torch.nn.functional.softmax(logpxobsgivenz + logpz - logq,0) # these are w_1,....,w_L for all observations in the batch
    xms = xgivenz.mean.reshape([L,batch_size,p])  # that's the only line that changed!
    xm=torch.einsum('ki,kij->ij', imp_weights, xms) 

    return xm

As well as the MSE function:

In [18]:
def mse(xhat,xtrue,mask): # MSE function for imputations
    xhat = np.array(xhat)
    xtrue = np.array(xtrue)
    return np.mean(np.power(xhat-xtrue,2)[~mask])

We instantiate the model using last updated federated parameters:

In [19]:
# extract federated model into PyTorch framework
model = exp.model_instance()
model.load_state_dict(exp.aggregated_params()[rounds - 1]['params'])

encoder = model.encoder
decoder = model.decoder

Same for the models trained with FedProx and FedCos

In [20]:
model_fedprox = exp_fedprox.model_instance()
model_fedprox.load_state_dict(exp_fedprox.aggregated_params()[rounds - 1]['params'])

encoder_fedprox = model_fedprox.encoder
decoder_fedprox = model_fedprox.decoder

model_fedcos = exp_fedcos.model_instance()
# We should remove the 'disp_global' key from the aggregated paramters since this
# is not needed to instantiate the model (just needed for training)
aggregated_params_fedcos = exp_fedcos.aggregated_params()[rounds - 1]['params']
del aggregated_params_fedcos['disp_global']
model_fedcos.load_state_dict(aggregated_params_fedcos)

encoder_fedcos = model_fedcos.encoder
decoder_fedcos = model_fedcos.decoder

And we finally do the imputation and evaluate the corresponding imputation error through MSE for each federated model:

In [21]:
L = 100

xhat[~mask] = miwae_impute(encoder = encoder,decoder = decoder,iota_x = torch.from_numpy(xhat_0).float(),mask = torch.from_numpy(mask).float(),d = d,L= L).cpu().data.numpy()[~mask]
err_test_data = np.array([mse(xhat,xfull,mask)])
print('Imputation MSE of fed model on testing data %g' %err_test_data)
print('-----')

xhat[~mask] = miwae_impute(encoder = encoder_fedprox,decoder = decoder_fedprox,iota_x = torch.from_numpy(xhat_0).float(),mask = torch.from_numpy(mask).float(),d = d,L= L).cpu().data.numpy()[~mask]
err_test_data_fedprox = np.array([mse(xhat,xfull,mask)])
print('Imputation MSE of fed model (with fedprox) on testing data  %g' %err_test_data_fedprox)
print('-----')

xhat[~mask] = miwae_impute(encoder = encoder_fedcos,decoder = decoder_fedcos,iota_x = torch.from_numpy(xhat_0).float(),mask = torch.from_numpy(mask).float(),d = d,L= L).cpu().data.numpy()[~mask]
err_test_data_fedcos = np.array([mse(xhat,xfull,mask)])
print('Imputation MSE of fed model (with fedcos) on testing data  %g' %err_test_data_fedcos)
print('-----')

Imputation MSE of fed model on testing data 0.574807
-----
Imputation MSE of fed model (with fedprox) on testing data  0.581406
-----
Imputation MSE of fed model (with fedcos) on testing data  0.567933
-----


## 2. Testing on a client's dataset

We are now going to use the final federated model to impute missing data of client 1, which have been used for training:

In [22]:
# We first recover data (full and with missing entries) from client 1
data_client_1 = np.copy(Clients_data[0])
xfull_cl1 = np.copy(data_client_1)
xfull_cl1 = (xfull_cl1 - np.mean(xfull_cl1,0))/np.std(xfull_cl1,0)
xmiss_cl1 = np.copy(Clients_missing[0])

mask_cl1 = np.isfinite(xmiss_cl1) # binary mask that indicates which values are missing
xhat_0_cl1 = np.copy(xmiss_cl1)
xhat_0_cl1[np.isnan(xmiss_cl1)] = 0
xhat_cl1 = np.copy(xhat_0_cl1) # This will be out imputed data matrix

### Now we do the imputation

xhat_cl1[~mask_cl1] = miwae_impute(encoder = encoder,decoder = decoder, iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(),d = d,L= L).cpu().data.numpy()[~mask_cl1]
err_cl1_data = np.array([mse(xhat_cl1,xfull_cl1,mask_cl1)])
print('Imputation MSE of fed model on data from client 1  %g' %err_cl1_data)
print('-----')

xhat_cl1[~mask_cl1] = miwae_impute(encoder = encoder_fedprox,decoder = decoder_fedprox, iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(),d = d,L= L).cpu().data.numpy()[~mask_cl1]
err_cl1_data_fedprox = np.array([mse(xhat_cl1,xfull_cl1,mask_cl1)])
print('Imputation MSE of fed model (with fedprox) on data from client 1  %g' %err_cl1_data_fedprox)
print('-----')

xhat_cl1[~mask_cl1] = miwae_impute(encoder = encoder_fedcos,decoder = decoder_fedcos, iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(),d = d,L= L).cpu().data.numpy()[~mask_cl1]
err_cl1_data_fedcos = np.array([mse(xhat_cl1,xfull_cl1,mask_cl1)])
print('Imputation MSE of fed model (with fedcos) on data from client 1  %g' %err_cl1_data_fedcos)
print('-----')

Imputation MSE of fed model on data from client 1  0.593466
-----
Imputation MSE of fed model (with fedprox) on data from client 1  0.619039
-----
Imputation MSE of fed model (with fedcos) on data from client 1  0.59457
-----


## 3. Local training and testing on a client

Finally, we test the performance of the same model trained locally and tested on the dataset from client 1. We will use a total of `epochs`x`rounds` local epochs.

In [23]:
def miwae_loss(iota_x,mask,d):
    
    p_z = td.Independent(td.Normal(loc=torch.zeros(d),scale=torch.ones(d)),1)
    
    batch_size = iota_x.shape[0]
    out_encoder = encoder(iota_x)
    q_zgivenxobs = td.Independent(td.Normal(loc=out_encoder[..., :d],scale=torch.nn.Softplus()(out_encoder[..., d:(2*d)])),1)

    zgivenx = q_zgivenxobs.rsample([K])
    zgivenx_flat = zgivenx.reshape([K*batch_size,d])

    out_decoder = decoder(zgivenx_flat)
    all_means_obs_model = out_decoder[..., :p]
    all_scales_obs_model = torch.nn.Softplus()(out_decoder[..., p:(2*p)]) + 0.001
    all_degfreedom_obs_model = torch.nn.Softplus()(out_decoder[..., (2*p):(3*p)]) + 3

    data_flat = torch.Tensor.repeat(iota_x,[K,1]).reshape([-1,1])
    tiledmask = torch.Tensor.repeat(mask,[K,1])

    all_log_pxgivenz_flat = torch.distributions.StudentT(loc=all_means_obs_model.reshape([-1,1]),scale=all_scales_obs_model.reshape([-1,1]),df=all_degfreedom_obs_model.reshape([-1,1])).log_prob(data_flat)
    all_log_pxgivenz = all_log_pxgivenz_flat.reshape([K*batch_size,p])

    logpxobsgivenz = torch.sum(all_log_pxgivenz*tiledmask,1).reshape([K,batch_size])
    logpz = p_z.log_prob(zgivenx)
    logq = q_zgivenxobs.log_prob(zgivenx)

    neg_bound = -torch.mean(torch.logsumexp(logpxobsgivenz + logpz - logq,0))

    return neg_bound

We perform the local training:

In [25]:
# We recover again data (full and with missing entries) from client 1
data_client_1 = np.copy(Clients_data[0])
xfull_cl1 = np.copy(data_client_1)
xfull_cl1 = (xfull_cl1 - np.mean(xfull_cl1,0))/np.std(xfull_cl1,0)
xmiss_cl1 = np.copy(Clients_missing[0])

mask_cl1 = np.isfinite(xmiss_cl1) # binary mask that indicates which values are missing
xhat_0_cl1 = np.copy(xmiss_cl1)
xhat_0_cl1[np.isnan(xmiss_cl1)] = 0
xhat_cl1 = np.copy(xhat_0_cl1) # This will be out imputed data matrix

n_epochs_local = n_epochs*rounds
bs = 48 # batch size

encoder_cl1 = nn.Sequential(
    torch.nn.Linear(p, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, 2*d),  # the encoder will output both the mean and the diagonal covariance
)

decoder_cl1 = nn.Sequential(
    torch.nn.Linear(d, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, 3*p),  # the decoder will output both the mean, the scale, and the number of degrees of freedoms (hence the 3*p)
)

optimizer_cl1 = torch.optim.Adam(list(encoder_cl1.parameters()) + list(decoder_cl1.parameters()),lr=1e-3)

def weights_init(layer):
    if type(layer) == nn.Linear: torch.nn.init.orthogonal_(layer.weight)
        
encoder_cl1.apply(weights_init)
decoder_cl1.apply(weights_init)

for ep in range(1,n_epochs_local):
    perm = np.random.permutation(n) # We use the "random reshuffling" version of SGD
    batches_data = np.array_split(xhat_0_cl1[perm,], n/bs)
    batches_mask = np.array_split(mask_cl1[perm,], n/bs)
    for it in range(len(batches_data)):
        optimizer_cl1.zero_grad()
        encoder_cl1.zero_grad()
        decoder_cl1.zero_grad()
        b_data = torch.from_numpy(batches_data[it]).float()
        b_mask = torch.from_numpy(batches_mask[it]).float()
        loss = miwae_loss(iota_x = b_data,mask = b_mask, d = d)
        loss.backward()
        optimizer_cl1.step()
    if ep % rounds == 1:
        print('Epoch %g' %ep)
        print('MIWAE likelihood bound  %g' %(-np.log(K)-miwae_loss(iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(), d = d).cpu().data.numpy())) # Gradient step      

Epoch 1
MIWAE likelihood bound  -2.74871
Epoch 31
MIWAE likelihood bound  -2.74119
Epoch 61
MIWAE likelihood bound  -2.70127
Epoch 91
MIWAE likelihood bound  -2.80703
Epoch 121
MIWAE likelihood bound  -2.7642


And we do the imputation on the same dataset:

In [26]:
xhat_cl1[~mask_cl1] = miwae_impute(encoder = encoder_cl1, decoder = decoder_cl1, iota_x = torch.from_numpy(xhat_0_cl1).float(),mask = torch.from_numpy(mask_cl1).float(),d = d,L= L).cpu().data.numpy()[~mask_cl1]
err_local_cl1_data = np.array([mse(xhat_cl1,xfull_cl1,mask_cl1)])
print('Imputation MSE of local model on data from same client (cl 1)  %g' %err_local_cl1_data)
print('-----')

Imputation MSE of local model on data from same client (cl 1)  1.0338
-----


As well as the imputation on the external test dataset:

In [27]:
xhat[~mask] = miwae_impute(encoder = encoder_cl1,decoder = decoder_cl1,iota_x = torch.from_numpy(xhat_0).float(),mask = torch.from_numpy(mask).float(),d = d,L= L).cpu().data.numpy()[~mask]
err_local_cl1_test_data = np.array([mse(xhat,xfull,mask)])
print('Imputation MSE of local model on testing data %g' %err_local_cl1_test_data)
print('-----')

Imputation MSE of local model on testing data 1.03657
-----


## Comparison of obtained results:

In [28]:
from tabulate import tabulate

print('Imputation MSE on testing data')
print('-----')
data = [['FedAvg', err_test_data],
['FedProx', err_test_data_fedprox],
['FedCos', err_test_data_fedcos],
['Local (cl1)', err_local_cl1_test_data]]
print (tabulate(data, headers=["Model", "Mean Squared Error (\u2193)"]))
print('-----')
print('-----')
print('Imputation MSE on local data from client 1')
print('-----')
data = [['FedAvg', err_cl1_data],
['FedProx', err_cl1_data_fedprox],
['FedCos', err_cl1_data_fedcos],
['Local (cl1)', err_local_cl1_data]]
print (tabulate(data, headers=["Model", "Mean Squared Error (\u2193)"]))

Imputation MSE on testing data
-----
Model          Mean Squared Error (↓)
-----------  ------------------------
FedAvg                       0.574807
FedProx                      0.581406
FedCos                       0.567933
Local (cl1)                  1.03657
-----
-----
Imputation MSE on local data from client 1
-----
Model          Mean Squared Error (↓)
-----------  ------------------------
FedAvg                       0.593466
FedProx                      0.619039
FedCos                       0.59457
Local (cl1)                  1.0338


2022-05-16 17:01:39,345 fedbiomed INFO - CRITICAL
					 NODE node_2ad05852-b939-4f31-8c03-b6905bf7497e
					 MESSAGE: Node stopped in signal_handler, probably by user decision (Ctrl C)
-----------------------------------------------------------------
2022-05-16 17:01:41,048 fedbiomed INFO - CRITICAL
					 NODE node_cab4ad93-ef47-49fd-b0ec-bfafb71572dc
					 MESSAGE: Node stopped in signal_handler, probably by user decision (Ctrl C)
-----------------------------------------------------------------
2022-05-16 17:01:42,583 fedbiomed INFO - CRITICAL
					 NODE node_16062a6e-3512-4146-a89c-c379e90c9300
					 MESSAGE: Node stopped in signal_handler, probably by user decision (Ctrl C)
-----------------------------------------------------------------


As you can see, the federated model performs better than the local one!